In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


In [2]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 36.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [3]:
!pip install sk-video

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.9 MB/s eta 0:00:00


In [4]:
import shutil
import os
import cv2
import math
import numpy as np
import tensorflow as tf
import pandas as pd
import skvideo.io
import mediapipe as mp
import glob
import keras
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense

In [5]:
def mediapipe_detection(image,model):
    image = cv2.cv2tColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cv2tColor(image,cv2.COLOR_RGB2BGR)
    return image,results


In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

def pose_estimation(image,results):

        # 1. Draw face landmarks
        #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
        #                         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        #                         mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        #                         )

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(102,255,51), thickness=3, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(102,255,51), thickness=3, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(255,255,0), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                                 )

        return image

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose,lh,rh])

In [ ]:
os.makedirs("/content/drive/MyDrive/VietNam SignLanguage/Mediapipe")

In [ ]:
for dirpathed,dirnamed,file in os.walk("/content/drive/MyDrive/VietNam SignLanguage/train_test/train_dir_2"):
    for class_  in dirnamed:
      os.makedirs(os.path.join("/content/drive/MyDrive/VietNam SignLanguage/Mediapipe",class_))


In [ ]:
os.makedirs("/content/drive/MyDrive/VietNam SignLanguage/mp_results")

In [9]:
c=[]
for dirpathed,dirname,files in (os.walk("/content/drive/MyDrive/VietNam SignLanguage/train_test/train_dir_2")):
  for class_name in dirname:
    i=0
    for dirpath,dirname,files in os.walk(os.path.join(dirpathed,class_name)):
     print(dirpath)
     print(files)
     print(len(files))
     for k in files:
        i=i+1
     c.append(class_name)
     c.append(i)

/content/drive/MyDrive/VietNam SignLanguage/train_test/train_dir_2/Xanh la
['Xanhla_001_002_0.mp4', 'Xanhla_001_002_1.mp4', 'Xanhla_001_002_2.mp4', 'Xanhla_001_003_0.mp4', 'Xanhla_001_003_1.mp4', 'Xanhla_001_003_2.mp4', 'Xanhla_001_005_0.mp4', 'Xanhla_001_005_1.mp4', 'Xanhla_001_005_2.mp4', 'Xanhla_001_001_0.mp4', 'Xanhla_001_001_1.mp4', 'Xanhla_001_001_2.mp4', 'Xanhla_001_004_0.mp4', 'Xanhla_001_004_1.mp4', 'Xanhla_001_004_2.mp4', 'Xanhla_002_001_0.mp4', 'Xanhla_002_001_1.mp4', 'Xanhla_002_001_2.mp4', 'Xanhla_002_002_0.mp4', 'Xanhla_002_002_1.mp4', 'Xanhla_002_002_2.mp4', 'Xanhla_002_004_0.mp4', 'Xanhla_002_004_1.mp4', 'Xanhla_002_004_2.mp4', 'Xanhla_002_005_0.mp4', 'Xanhla_002_005_1.mp4', 'Xanhla_002_005_2.mp4', 'Xanhla_003_002_0.mp4', 'Xanhla_003_002_1.mp4', 'Xanhla_003_002_2.mp4', 'Xanhla_003_003_0.mp4', 'Xanhla_003_003_1.mp4', 'Xanhla_003_003_2.mp4', 'Xanhla_003_005_0.mp4', 'Xanhla_003_005_1.mp4', 'Xanhla_003_005_2.mp4', 'Xanhla_003_004_0.mp4', 'Xanhla_003_004_1.mp4', 'Xanhla_003_

In [10]:
Xanh_la = 200
Do = 200
Xanh_duong = 200
Hong = 200
Vang = 200

In [32]:
DATA_PATH = os.path.join('/content/drive/MyDrive/VietNam SignLanguage/mp_results',"Do")

#Actions
actions = np.array(['Xanh_la ','Do','Vang','Xanh_duong','Hong'])

#1179 videos worth of data
no_sequences = 1000

#45 frames
sequence_length = 10


In [16]:
for sequence in range(Do):
        try:
            os.makedirs(os.path.join(DATA_PATH,str(sequence)))
        except:
            pass

In [20]:
DATA_PATH = os.path.join('/content/drive/MyDrive/VietNam SignLanguage/mp_results',"Vang")
for sequence in range(Do):
        try:
            os.makedirs(os.path.join(DATA_PATH,str(sequence)))
        except:
            pass

In [21]:
DATA_PATH = os.path.join('/content/drive/MyDrive/VietNam SignLanguage/mp_results',"Xanh la")
for sequence in range(Do):
        try:
            os.makedirs(os.path.join(DATA_PATH,str(sequence)))
        except:
            pass

In [22]:
DATA_PATH = os.path.join('/content/drive/MyDrive/VietNam SignLanguage/mp_results',"Hong")
for sequence in range(Do):
        try:
            os.makedirs(os.path.join(DATA_PATH,str(sequence)))
        except:
            pass

In [23]:
DATA_PATH = os.path.join('/content/drive/MyDrive/VietNam SignLanguage/mp_results',"Xanh Duong")
for sequence in range(Do):
        try:
            os.makedirs(os.path.join(DATA_PATH,str(sequence)))
        except:
            pass

In [24]:
for dirpathed,dirnamed,file in os.walk("/content/drive/MyDrive/VietNam SignLanguage/Mediapipe"):
    for class_  in dirnamed:
      os.makedirs(os.path.join("/content/drive/MyDrive/VietNam SignLanguage/Output",class_))